In [4]:
import json
from IPython.display import display, Markdown

# Define parameters.

PROMPTS = "../data/prompts/apihtml/taskvine_apihtml_gpt4omini_1_analysis.json"

In [5]:
# Analyze a singular prompt.

def analyze(prompt: dict):
    print(prompt["content"])
    display(Markdown(prompt["response"]))
    return input("Analysis: ")

In [6]:
# Load prompts from file.
with open(PROMPTS, "r") as fp:
    prompts = json.load(fp)

# Perform the zeroshot prompting.
for prompt in prompts["prompts"]:
    if "analysis" not in prompt or input("Override analysis? (y/n)").lower() in ["y", "yes"]:
        analysis = analyze(prompt)
        prompt["analysis"] = analysis

Write a Python TaskVine application to get the current date on a worker machine. The date must be stored to an output file named "current_date".


```python
from ndcctools.taskvine import Task, Manager

# Define the command to get the current date
command = "date > current_date"

# Create a Task object
task = Task(command)

# Create a Manager object
manager = Manager()

# Submit the task to the manager
manager.submit(task)

# Wait for the task to complete
manager.wait()

# Print the result
print("Task completed. The current date is stored in 'current_date'.")
```

Analysis:  q


Write a Python TaskVine application to count the number of times the keywords "wealth", "nation", "labour", "price", "nature", and "commodity" appear in Adam Smith's Wealth of Nations. The text is available at "https://www.gutenberg.org/cache/epub/3300/pg3300.txt".


```python
import requests
from ndcctools.taskvine import Manager

# Define the keywords to search for
keywords = ["wealth", "nation", "labour", "price", "nature", "commodity"]

def count_keywords(text):
    """Count occurrences of each keyword in the text."""
    counts = {keyword: text.lower().count(keyword) for keyword in keywords}
    return counts

def main():
    # Fetch the text from the URL
    url = "https://www.gutenberg.org/cache/epub/3300/pg3300.txt"
    response = requests.get(url)
    text = response.text

    # Count the keywords
    keyword_counts = count_keywords(text)

    # Print the results
    for keyword, count in keyword_counts.items():
        print(f"The keyword '{keyword}' appears {count} times.")

if __name__ == "__main__":
    # Create a TaskVine manager
    manager = Manager()
    
    # Submit the main function as a task
    manager.submit(main)
```

This code defines a Python TaskVine application that fetches the text of "Wealth of Nations" from the provided URL, counts the occurrences of specified keywords, and prints the results. The `count_keywords` function performs the counting, and the `main` function orchestrates the fetching and counting process. The application is submitted to the TaskVine manager for execution.

...

```python
import requests
from ndcctools.taskvine import manager, task
from ndcctools.taskvine.futures import Future

# Define a function to fetch the title of a webpage
def fetch_title(url):
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses
    title_start = response.text.find('<title>') + len('<title>')
    title_end = response.text.find('</title>', title_start)
    title = response.text[title_start:title_end]
    return url, title

# Create a TaskVine manager
manager_instance = manager.Manager()

# List of URLs to fetch
urls = [
    "https://abcnews.com",
    "https://apnews.com",
    "https://axios.com",
    "https://cnn.com",
    "https://nbcnews.com",
    "https://nytimes.com",
    "https://reuters.com",
    "https://washingtonpost.com",
    "https://wsj.com"
]

# Create a list to hold future tasks
futures = []

# Submit tasks to fetch titles asynchronously
for url in urls:
    task_instance = task.Task(fetch_title, url)
    future = manager_instance.submit(task_instance)
    futures.append(future)

# Wait for tasks to complete and print results in order of completion
for future in futures:
    result = future.wait()
    print(f"URL: {result[0]}, Title: {result[1]}")
```

Analysis:  q


Write a Python TaskVine application to get the current date on a worker machine. The date must be stored to an output file named "current_date". To prevent an unwanted worker from connecting to the manager, use the password stored in the file "taskvine.password". Further, encrypt communication between the manager and worker with SSL using the "key.pem" and "certificate.pem" files.


```python
from ndcctools.taskvine import manager, task

# Define the command to get the current date and store it in a file
command = "date > current_date"

# Create a Task object
date_task = task.Task(command)

# Create a Manager object with SSL and password protection
manager_instance = manager.Manager(
    port=0,  # Random port
    name="DateFetcher",
    shutdown=False,
    run_info_path="vine-run-info",
    ssl=("key.pem", "certificate.pem"),
)

# Set the password for the manager from the file
with open("taskvine.password", "r") as password_file:
    password = password_file.read().strip()
manager_instance.set_password(password)

# Submit the task to the manager
manager_instance.submit(date_task)

# Wait for the task to complete
manager_instance.wait()
```

Analysis:  q
